# 飞行器模型搭建

## 回放缓冲区
大多数现代强化学习算法都使用一个回放存储器或缓冲区来**存储**和**回调**经验元组。

In [0]:
import random
from collections import namedtuple, deque

class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, buffer_size, batch_size):
        """Initialize a ReplayBuffer object.
        Params
        ======
            buffer_size: maximum size of buffer
            batch_size: size of each training batch
        """
        self.memory = deque(maxlen=buffer_size)  # internal memory (deque)
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])

    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)

    def sample(self, batch_size=64):
        """Randomly sample a batch of experiences from memory."""
        return random.sample(self.memory, k=self.batch_size)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

## 深度确定性策略梯度 (DDPG)
你可以使用很多种不同的算法来设计智能体，只要它适合连续状态和动作空间即可。一种热门方法是深度确定性策略梯度，简称 DDPG。它实际上是一种行动者-评论者方法，但是关键在于底层的策略函数本身确定性函数，从外部添加了一些噪点，以便采取的动作具有理想的随机性。

我们来实现原始论文中给出的算法：

Lillicrap, Timothy P等，2015. [深度强化学习连续控制](https://arxiv.org/pdf/1509.02971.pdf).

可以使用最现代的深度学习库（例如 Keras 或 TensorFlow）实现该算法的两大组件 - 行动者和评论者网络。





### DDPG：行动者（策略）模型
以下是使用 Keras 定义的一个非常简单的行动者模型。

In [0]:
from keras import layers, models, optimizers
from keras import backend as K

class Actor:
    """Actor (Policy) Model."""

    def __init__(self, state_size, action_size, action_low, action_high):
        """Initialize parameters and build model.

        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            action_low (array): Min value of each action dimension
            action_high (array): Max value of each action dimension
        """
        self.state_size = state_size
        self.action_size = action_size
        self.action_low = action_low
        self.action_high = action_high
        self.action_range = self.action_high - self.action_low

        # Initialize any other variables here

        self.build_model()

    def build_model(self):
        """Build an actor (policy) network that maps states -> actions."""
        # Define input layer (states)
        states = layers.Input(shape=(self.state_size,), name='states')

        # Add hidden layers
        net = layers.Dense(units=32, activation='relu')(states)
        net = layers.Dense(units=64, activation='relu')(net)
        net = layers.Dense(units=32, activation='relu')(net)

        # Try different layer sizes, activations, add batch normalization, regularizers, etc.

        # Add final output layer with sigmoid activation
        raw_actions = layers.Dense(units=self.action_size, activation='sigmoid',
            name='raw_actions')(net)

        # Scale [0, 1] output for each action dimension to proper range
        actions = layers.Lambda(lambda x: (x * self.action_range) + self.action_low,
            name='actions')(raw_actions)

        # Create Keras model
        self.model = models.Model(inputs=states, outputs=actions)

        # Define loss function using action value (Q value) gradients
        action_gradients = layers.Input(shape=(self.action_size,))
        loss = K.mean(-action_gradients * actions)

        # Incorporate any additional losses here (e.g. from regularizers)

        # Define optimizer and training function
        optimizer = optimizers.Adam()
        updates_op = optimizer.get_updates(params=self.model.trainable_weights, loss=loss)
        self.train_fn = K.function(
            inputs=[self.model.input, action_gradients, K.learning_phase()],
            outputs=[],
            updates=updates_op)

注意，输出层生成的原始动作位于[0.0, 1.0]范围内（使用 sigmoid 激活函数）。因此，我们添加另一个层级，该层级会针对每个动作维度将每个输出缩放到期望的范围。这样会针对任何给定状态向量生成确定性动作。稍后将向此动作添加噪点，以生成某个探索性行为。

另一个需要注意的是损失函数如何使用动作值（Q 值）梯度进行定义：

```
# Define loss function using action value (Q value) gradients
action_gradients = layers.Input(shape=(self.action_size,))
loss = K.mean(-action_gradients * actions)
```

这些梯度需要使用评论者模型计算，并在训练时提供梯度。因此指定为在训练函数中使用的“输入”的一部分：


```
self.train_fn = K.function(
   inputs=[self.model.input, action_gradients, K.learning_phase()],
    outputs=[],
    updates=updates_op)
```



### DDPG: 评论者模型
相应的评论者模型可以如下所示：

In [0]:
class Critic:
    """Critic (Value) Model."""

    def __init__(self, state_size, action_size):
        """Initialize parameters and build model.

        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
        """
        self.state_size = state_size
        self.action_size = action_size

        # Initialize any other variables here

        self.build_model()

    def build_model(self):
        """Build a critic (value) network that maps (state, action) pairs -> Q-values."""
        # Define input layers
        states = layers.Input(shape=(self.state_size,), name='states')
        actions = layers.Input(shape=(self.action_size,), name='actions')

        # Add hidden layer(s) for state pathway
        net_states = layers.Dense(units=32, activation='relu')(states)
        net_states = layers.Dense(units=64, activation='relu')(net_states)

        # Add hidden layer(s) for action pathway
        net_actions = layers.Dense(units=32, activation='relu')(actions)
        net_actions = layers.Dense(units=64, activation='relu')(net_actions)

        # Try different layer sizes, activations, add batch normalization, regularizers, etc.

        # Combine state and action pathways
        net = layers.Add()([net_states, net_actions])
        net = layers.Activation('relu')(net)

        # Add more layers to the combined network if needed

        # Add final output layer to prduce action values (Q values)
        Q_values = layers.Dense(units=1, name='q_values')(net)

        # Create Keras model
        self.model = models.Model(inputs=[states, actions], outputs=Q_values)

        # Define optimizer and compile model for training with built-in loss function
        optimizer = optimizers.Adam()
        self.model.compile(optimizer=optimizer, loss='mse')

        # Compute action gradients (derivative of Q values w.r.t. to actions)
        action_gradients = K.gradients(Q_values, actions)

        # Define an additional function to fetch action gradients (to be used by actor model)
        self.get_action_gradients = K.function(
            inputs=[*self.model.input, K.learning_phase()],
            outputs=action_gradients)

它在某些方面比行动者模型简单，但是需要注意几点。首先，行动者模型旨在将状态映射到动作，而评论者模型需要将（状态、动作）对映射到它们的 Q 值。这一点体现在了输入层中。



```
# Define input layers
states = layers.Input(shape=(self.state_size,), name='states')
actions = layers.Input(shape=(self.action_size,), name='actions')

```
这两个层级首先可以通过单独的“路径”（迷你子网络）处理，但是最终需要结合到一起。例如，可以通过使用 Keras 中的 Add 层级类型来实现请参阅[合并层级](https://keras.io/layers/merge/)):




```
# Combine state and action pathways
net = layers.Add()([net_states, net_actions])
```


该模型的最终输出是任何给定（状态、动作）对的 Q 值。但是，我们还需要计算此 Q 值相对于相应动作向量的梯度，以用于训练行动者模型。这一步需要明确执行，并且需要定义一个单独的函数来访问这些梯度：

```
# Compute action gradients (derivative of Q values w.r.t. to actions)
action_gradients = K.gradients(Q_values, actions)

# Define an additional function to fetch action gradients (to be used by actor model)
self.get_action_gradients = K.function(
    inputs=[*self.model.input, K.learning_phase()],
    outputs=action_gradients)
```

### DDPG: 智能体
现在我们可以将行动者模型和策略模型放到一起，构建 DDPG 智能体了。注意，我们需要每个模型的两个副本，一个本地副本，一个目标副本。该技巧来自深度 Q 学习中的“固定 Q 目标”，用于拆分被更新的参数和生成目标值的参数。

以下是智能体类的纲要：

In [0]:
class DDPG():
    """Reinforcement Learning agent that learns using DDPG."""
    def __init__(self, task):
        self.task = task
        self.state_size = task.state_size
        self.action_size = task.action_size
        self.action_low = task.action_low
        self.action_high = task.action_high

        # Actor (Policy) Model
        self.actor_local = Actor(self.state_size, self.action_size, self.action_low, self.action_high)
        self.actor_target = Actor(self.state_size, self.action_size, self.action_low, self.action_high)

        # Critic (Value) Model
        self.critic_local = Critic(self.state_size, self.action_size)
        self.critic_target = Critic(self.state_size, self.action_size)

        # Initialize target model parameters with local model parameters
        self.critic_target.model.set_weights(self.critic_local.model.get_weights())
        self.actor_target.model.set_weights(self.actor_local.model.get_weights())

        # Noise process
        self.exploration_mu = 0
        self.exploration_theta = 0.15
        self.exploration_sigma = 0.2
        self.noise = OUNoise(self.action_size, self.exploration_mu, self.exploration_theta, self.exploration_sigma)

        # Replay memory
        self.buffer_size = 100000
        self.batch_size = 64
        self.memory = ReplayBuffer(self.buffer_size, self.batch_size)

        # Algorithm parameters
        self.gamma = 0.99  # discount factor
        self.tau = 0.01  # for soft update of target parameters

    def reset_episode(self):
        self.noise.reset()
        state = self.task.reset()
        self.last_state = state
        return state

    def step(self, action, reward, next_state, done):
         # Save experience / reward
        self.memory.add(self.last_state, action, reward, next_state, done)

        # Learn, if enough samples are available in memory
        if len(self.memory) > self.batch_size:
            experiences = self.memory.sample()
            self.learn(experiences)

        # Roll over last state and action
        self.last_state = next_state

    def act(self, states):
        """Returns actions for given state(s) as per current policy."""
        state = np.reshape(states, [-1, self.state_size])
        action = self.actor_local.model.predict(state)[0]
        return list(action + self.noise.sample())  # add some noise for exploration

    def learn(self, experiences):
        """Update policy and value parameters using given batch of experience tuples."""
        # Convert experience tuples to separate arrays for each element (states, actions, rewards, etc.)
        states = np.vstack([e.state for e in experiences if e is not None])
        actions = np.array([e.action for e in experiences if e is not None]).astype(np.float32).reshape(-1, self.action_size)
        rewards = np.array([e.reward for e in experiences if e is not None]).astype(np.float32).reshape(-1, 1)
        dones = np.array([e.done for e in experiences if e is not None]).astype(np.uint8).reshape(-1, 1)
        next_states = np.vstack([e.next_state for e in experiences if e is not None])

        # Get predicted next-state actions and Q values from target models
        #     Q_targets_next = critic_target(next_state, actor_target(next_state))
        actions_next = self.actor_target.model.predict_on_batch(next_states)
        Q_targets_next = self.critic_target.model.predict_on_batch([next_states, actions_next])

        # Compute Q targets for current states and train critic model (local)
        Q_targets = rewards + self.gamma * Q_targets_next * (1 - dones)
        self.critic_local.model.train_on_batch(x=[states, actions], y=Q_targets)

        # Train actor model (local)
        action_gradients = np.reshape(self.critic_local.get_action_gradients([states, actions, 0]), (-1, self.action_size))
        self.actor_local.train_fn([states, action_gradients, 1])  # custom training function

        # Soft-update target models
        self.soft_update(self.critic_local.model, self.critic_target.model)
        self.soft_update(self.actor_local.model, self.actor_target.model)   

    def soft_update(self, local_model, target_model):
        """Soft update model parameters."""
        local_weights = np.array(local_model.get_weights())
        target_weights = np.array(target_model.get_weights())

        assert len(local_weights) == len(target_weights), "Local and target model parameters must have the same size"

        new_weights = self.tau * local_weights + (1 - self.tau) * target_weights
        target_model.set_weights(new_weights)

注意，在用多个经验进行训练后，我们可以将新学习的权重（来自本地模型）复制到目标模型中。但是，单个批次可能会向该流程中引入很多偏差，因此最后进行软更新，由参数 tau 控制。

最后还要完成一步，才能使整个项目能正常运行，你需要创建合适的噪点模型。

## Ornstein–Uhlenbeck 噪点
我们将使用一个具有所需特性的特定噪点流程，称之为 [Ornstein–Uhlenbeck 流程](https://en.wikipedia.org/wiki/Ornstein%E2%80%93Uhlenbeck_process)。它会根据高斯（正态）分布生成随机样本，但是每个样本都会影响到后续样本，使两个连续样本很接近。因此本质上是马尔可夫流程。

为何与我们有关呢？我们可以直接从高斯分布中抽样吧？是的，但是注意，我们希望使用该流程向动作中添加一些噪点，以便促进探索性行为。因为动作就是应用到飞行器上的力和扭矩，我们希望连续动作不要变化太大。否则，我们可能不会飞到任何地方！想象下随机上下左右翻转控制器！

除了样本的暂时相关特性外，QU 流程的另一个好处是在一段时间之后，它会逐渐接近指定的均值。在用它生成噪点时，我们可以指定均值为 0，当我们在学习任务上取得进展时，它将能够减少探索行为。

以下是一个你可以采用的示例 Ornstein-Uhlenbeck 流程实现。

In [0]:
class OUNoise:
    """Ornstein-Uhlenbeck process."""

    def __init__(self, size, mu=None, theta=0.15, sigma=0.3):
        """Initialize parameters and noise process."""
        self.size = size
        self.mu = mu if mu is not None else np.zeros(self.size) #此处需修改mu
        self.theta = theta
        self.sigma = sigma
        self.state = np.ones(self.size) * self.mu
        self.reset()

    def reset(self):
        """Reset the internal state (= noise) to mean (mu)."""
        self.state = self.mu

    def sample(self):
        """Update internal state and return it as a noise sample."""
        x = self.state
        dx = self.theta * (self.mu - x) + self.sigma * np.random.randn(len(x))
        self.state = x + dx
        return self.state

### 我的飞行器需要训练多久？
根据随机的初始参数，有时你的飞行器会在初始的20-50个迭代循环里习得你的任务，但有时你会可能需要这个训练次数达到500-1000次。此外，你的飞行器也可能陷入局部最优解，而无法达到更优的学习效果。因此，你的飞行器可能需要很长时间来完成你的任务，这个主要依赖于你选择的学习率learning rate。

如果你发现你的飞行器学习了1500-2000次仍然没有显著进展，你可以先让你的飞行器先试试简单的连续动作问题，如 [Mountain Car](https://gym.openai.com/envs/MountainCarContinuous-v0/) 或者 [Pendulum](https://gym.openai.com/envs/Pendulum-v0/) 。当然，你的模型大小会随着你状态/行动空间的不同定义而改变，但是至少可以保障你的算法实现。

其次，调整飞行器的状态以及/或者动作空间定义，来保障你的飞行器拥有足够的学习参数（但不要过多！）最后（这是许多最多迭代的部分），尝试调整奖励函数来让你的飞行器得到一个更具体的学习目的。调整矩阵里的**权重**，增加额外的奖励/惩罚，尝试一个**非线性映射矩阵**。同时，你可能也需要将**奖惩的值标准化**，如从-1.0到1.0。这样的转换将避免梯度爆炸造成的不稳定性。

### 我的飞行器的确在学习，但是即便是过了很久，依然没有达到我的要求。我该怎么做？
想要让一个强化学习智能体学习你_实际_想要它学的，是十分困难的，也十分费时。你的飞行器只会根据你定义的奖励函数来习得最优的策略，但是你的奖励函数可能没有包含你期待的所有方面，比如飞行任务等。如果你没有预设你的飞行器做一些转体，只是关心它飞多高，那它就很可能会旋转上升！

有时，你的算法可能没有与环境交互好，如，可能部分环境需要额外的探索及噪音，而部分环境则可以减少噪音。你可以试试改变一些变量，来看会不会优化结果。但是即使你已经有了一个很优秀的算法，你都可能需要几天或几周来习得一个复杂的任务！

这就是为什么我们希望你绘制一个奖励曲线。只要每次迭代的平均奖励是总体上升的（即便存在一些噪音），就可以了。飞行最后的行为展示不需要非常完美。

### 我可以使用一个深度Q网络来解决问题吗？
当然！但是...记得一个深度Q网络（DQN）意味着解决一个离散动作空间问题，然而我们的飞行器控制问题是一个连续动作域问题。因此我们并不建议一个离散空间算法，但是你可以仍旧使用这个DQN来映射最终的输出结果至一个适当的连续动作空间。尽管这个过程可能让神经网络难以理解动作之间的互相关联，但是它可能也会简化这个训练算法。